# Pytorch 로  RNN 구현 실습하기

In [2]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 484.0MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b7cc000 @  0x7feca3b8e1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 8.8MB/s 
    100% |████████████████████████████████| 2.0MB 18.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [3]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(1) 

지난번과 같이  MNIST 데이터를 불러옵니다. 

In [0]:
# Hyper Parameters
EPOCH = 1              
BATCH_SIZE = 64
TIME_STEP = 28        
INPUT_SIZE = 28         # rnn input size / image width
LR = 0.01               # learning rate
DOWNLOAD_MNIST = True 

In [6]:
# Mnist digital dataset
train_data = dsets.MNIST(
    root='./mnist/',
    train=True,                        
    transform=transforms.ToTensor(),    
                                        # torch.FloatTensor of shape (C x H x W) and normalize in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,          
)

Processing...
Done!


In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_data = dsets.MNIST(root='./mnist/', train=False, transform=transforms.ToTensor())
test_x = Variable(test_data.test_data, volatile=True).type(torch.FloatTensor)[:2000]/255.   # shape (2000, 28, 28) value in range(0,1)
test_y = test_data.test_labels.numpy().squeeze()[:2000] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


 Create a RNN model


lstm 모듈에는 3차원의  input size 를 넣어주어야 합니다.  batch_first=True 를 하지 않으면 첫번째  dim  은  sequence , 2nd 는  batch_size, 3rd 는   각 sequence 의  element  들을 넣어줍니다.




In [0]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.LSTM ( input_size = INPUT_SIZE, # input  크기
                           hidden_size = 64, # RNN 의  hidden state size
                            num_layers=1, # RNN layer 개수
                             batch_first=True, # input, output 의 첫번쨰 dimension 이  batch
                           )
        
        self.out = nn.Linear(64,10)
        
    def forward(self, x): # forward 함수
      #  x 는  batch, time_step, input_size 크기
      #  r_out 은  batch, time_step, output_size
      # h_n, h_c  는  n_layer, batch, hidden_size -   hidden state 와  cell state
      r_out, (h_n, h_c) = self.rnn(x, None)
      out = self.out(r_out[:, -1, :])
      return out

In [9]:
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [12]:
# 아래는 똑같음.
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss() 

for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):        # gives batch data
        b_x = Variable(x.view(-1, 28, 28))              # reshape x to (batch, time_step, input_size)
        b_y = Variable(y)                               # batch y

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        if step % 50 == 0:
            test_output = rnn(test_x)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
            accuracy = sum(pred_y == test_y) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch:  0 | train loss: 0.0798 | test accuracy: 0.96


KeyboardInterrupt: ignored

참고자료 

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

https://github.com/MorvanZhou/PyTorch-Tutorial/blob/master/tutorial-contents-notebooks/402_RNN.ipynb
